In [21]:
import open3d as o3d
import numpy as np

LASER_PATH = "pointclouds/accumulated_laserscan_scene7.ply"
MAP_PATH = "../../labelCloud/pointclouds/cloud_scene7.ply"

laser_pcd = o3d.io.read_point_cloud(LASER_PATH)
map_pcd = o3d.io.read_point_cloud(MAP_PATH)

laser_pcd.paint_uniform_color([1, 0, 0])
#map_pcd.paint_uniform_color([0, 1, 0])

o3d.visualization.draw_geometries([laser_pcd, map_pcd])

In [15]:
uniques_values = np.unique(np.asarray(laser_pcd.points, dtype=np.float16)[:, 2])
uniques_values

array([0.2], dtype=float16)

In [17]:
z = np.array(laser_pcd.points)[0, 2]
slack = 0.01

map_points = np.array(map_pcd.points)
map_points = map_points[map_points[:, 2] >= z - slack]
map_points = map_points[map_points[:, 2] <= z + slack]
#map_points[:, 2] = z

map_pcd.points = o3d.utility.Vector3dVector(map_points)
map_pcd.paint_uniform_color([0, 1, 0])

o3d.visualization.draw_geometries([laser_pcd, map_pcd])

In [20]:
from scipy.spatial import KDTree
import sensor_msgs.point_cloud2 as pc2

def compute_distances(source_points, target_points):
    tree = KDTree(target_points)
    distances, _ = tree.query(source_points)
    return distances


laser_scan_points = np.asarray(laser_pcd.points)
camera_points = np.asarray(map_pcd.points)

# Compute nearest neighbor distances from camera points to laser scan points
distances = compute_distances(camera_points, laser_scan_points)

# Compute statistics
mean_distance = np.mean(distances)
median_distance = np.median(distances)
std_distance = np.std(distances)

print(f"Mean Distance: {mean_distance}")
print(f"Median Distance: {median_distance}")
print(f"Standard Deviation: {std_distance}")


Mean Distance: 0.05751469240990484
Median Distance: 0.035514221243927696
Standard Deviation: 0.06481701338712968
